## EDA - Exploratory Data Analysis of drug_review data

In [ ]:
# =====================
# 1. Imports
# =====================
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("oladayoowoeye/drug-review-prediction-sentiment-analysis")

print("Path to dataset files:", path)